In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
import cv2
import numpy as np
from scipy.ndimage import gaussian_filter
from tqdm import tqdm

def add_cracks(image, num_cracks=5, crack_width=2):
    result = image.copy()
    height, width = image.shape[:2]
    for _ in range(num_cracks):
        start_point = (np.random.randint(0, width), np.random.randint(0, height))
        end_point = (np.random.randint(0, width), np.random.randint(0, height))
        color = (np.random.randint(0, 50), np.random.randint(0, 50), np.random.randint(0, 50))
        cv2.line(result, start_point, end_point, color, crack_width)
    return result

def add_color_fading(image, fade_factor=0.7):
    return cv2.addWeighted(image, fade_factor, np.ones_like(image) * 255, 1 - fade_factor, 0)

def add_scratches(image, num_scratches=10, max_length=50):
    result = image.copy()
    height, width = image.shape[:2]
    for _ in range(num_scratches):
        start_point = (np.random.randint(0, width), np.random.randint(0, height))
        angle = np.random.uniform(0, 2 * np.pi)
        length = np.random.randint(10, max_length)
        end_point = (
            int(start_point[0] + length * np.cos(angle)),
            int(start_point[1] + length * np.sin(angle))
        )
        cv2.line(result, start_point, end_point, (200, 200, 200), 1)
    return result

def add_water_damage(image, num_spots=5, max_radius=50):
    result = image.copy()
    height, width = image.shape[:2]
    for _ in range(num_spots):
        center = (np.random.randint(0, width), np.random.randint(0, height))
        radius = np.random.randint(10, max_radius)
        color = (np.random.randint(150, 200), np.random.randint(150, 200), np.random.randint(150, 200))
        cv2.circle(result, center, radius, color, -1)
        result = cv2.addWeighted(result, 0.7, image, 0.3, 0)
    return result

def add_mold(image, num_spots=10, max_radius=20):
    result = image.copy()
    height, width = image.shape[:2]
    for _ in range(num_spots):
        center = (np.random.randint(0, width), np.random.randint(0, height))
        radius = np.random.randint(5, max_radius)
        color = (np.random.randint(0, 100), np.random.randint(50, 150), np.random.randint(0, 100))
        cv2.circle(result, center, radius, color, -1)
    return result

def add_noise(image, mean=0, std=25):
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    return cv2.add(image, noise)

def apply_damage_effects(image):
    damaged_image = image.copy()

    # 랜덤하게 효과 선택 및 적용
    if np.random.rand() < 0.3:
        damaged_image = add_cracks(damaged_image, num_cracks=np.random.randint(3, 8))
    if np.random.rand() < 0.4:
        damaged_image = add_color_fading(damaged_image, fade_factor=np.random.uniform(0.6, 0.9))
    if np.random.rand() < 0.6:
        damaged_image = add_scratches(damaged_image, num_scratches=np.random.randint(5, 15))
    if np.random.rand() < 0.3:
        damaged_image = add_water_damage(damaged_image, num_spots=np.random.randint(2, 6))
    if np.random.rand() < 0.5:
        damaged_image = add_mold(damaged_image, num_spots=np.random.randint(5, 15))
    if np.random.rand() < 0.4:
        damaged_image = add_noise(damaged_image, std=np.random.randint(10, 30))

    return damaged_image

def process_images(input_dir, output_dir):
    # 출력 디렉토리가 없으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 입력 디렉토리의 모든 jpg 파일 처리
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.jpg')]

    for filename in tqdm(image_files, desc="Processing images"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, f"damaged_{filename}")

        # 이미지 읽기
        image = cv2.imread(input_path)

        if image is None:
            print(f"Failed to read image: {input_path}")
            continue

        # 손상 효과 적용
        damaged_image = apply_damage_effects(image)

        # 손상된 이미지 저장
        cv2.imwrite(output_path, damaged_image)

# 메인 실행 부분
input_directory = "/content/drive/MyDrive/KeyPoint/images3"
output_directory = "/content/drive/MyDrive/KeyPoint/images4"

process_images(input_directory, output_directory)
print("Image processing completed.")

Processing images: 100%|██████████| 300/300 [00:09<00:00, 33.25it/s]

Image processing completed.
